Gradient Boosting ~ Jyosna Philip

In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,root_mean_squared_error #Additional scklearn functions
from sklearn.model_selection import GroupShuffleSplit,GridSearchCV

In [48]:
x_train = pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\XTrain.csv")
y_train = pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\YTrain.csv")
engine = np.array(pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\Engine.csv"))

In [49]:
Engine=engine.ravel()

gradient boosting -base model

In [40]:
gss_b = GroupShuffleSplit(n_splits = 1, train_size = 0.7, random_state = 42)
train_index_b,val_index_b=next(gss_b.split(x_train, y_train, engine))
Y_train=np.asarray(y_train).ravel()
gb_b =GradientBoostingRegressor(random_state=42)
gb_b.fit(x_train.iloc[train_index_b,:], Y_train[train_index_b])
prediction=gb_b.predict(x_train.iloc[val_index_b,:])
print(r2_score( Y_train[val_index_b],prediction))
print(root_mean_squared_error( Y_train[val_index_b],prediction))

0.6891472768126499
36.330892296130735


0.6891472768126499
36.330892296130735

gradient boosting with cv

In [41]:
gss_outer = GroupShuffleSplit(n_splits = 10, train_size = 0.7, random_state = 42) #n_splits refers to number of folds required
r2=[]
rmse=[]
for i, (train_index, val_index) in enumerate(gss_outer.split(x_train, y_train, Engine)):
    xtrain = x_train.iloc[train_index, :] # Train Data from training set (70% split)
    ytrain = y_train.iloc[train_index, :]
    Y_train=np.asarray(ytrain).ravel()
    xval = x_train.iloc[val_index, :] # Validation Data from training set (30% split)
    yval = y_train.iloc[val_index, :]
    
    gb =GradientBoostingRegressor(random_state=42)
    gb.fit(xtrain, Y_train)
    r2.append(r2_score( yval,gb.predict(xval)))
    rmse.append(root_mean_squared_error(yval,gb.predict(xval)))
print('R2: %.3f (%.3f)' % (np.mean(r2), np.std(r2)))
print('RMSE: %.3f (%.3f)' % (np.mean(rmse), np.std(rmse)))


R2: 0.652 (0.046)
RMSE: 40.262 (3.484)


R2: 0.652 (0.046)
RMSE: 40.262 (3.484)

gradient boosting with gridsearch

In [51]:
gss_grid =  GroupShuffleSplit(n_splits = 1, train_size = 0.7, random_state = 42)

gb_grid = GradientBoostingRegressor(random_state=42,verbose=1)
para={"learning_rate":[0.05,0.1,0.2],"n_estimators":[70,100,200,300],"max_depth":[5,6,7,8],"min_samples_split":[150,200],"criterion":["squared_error"],"min_samples_leaf":[50,80] }
train_index_grid,val_index_grid=next(gss_grid.split(x_train, y_train, Engine))
# define search
search = GridSearchCV(gb_grid, param_grid=para, scoring=['r2','neg_root_mean_squared_error'], cv=gss_grid, refit='r2')
# execute search
Y_train=np.asarray(y_train).ravel()
result = search.fit(x_train.iloc[train_index_grid,:], Y_train[train_index_grid],groups=Engine[train_index_grid])
# get the best performing model fit on the whole training set
best_model = result.best_estimator_
yhat = best_model.predict(x_train[val_index_grid,:])
# evaluate the model
r2 = r2_score(Y_train[val_index_grid], yhat)
rmse=root_mean_squared_error(Y_train[val_index_grid],yhat)
print("r2: ",r2)
print("rmse: ",rmse)

      Iter       Train Loss   Remaining Time 
         1        3819.0255            7.86s
         2        3529.5422            6.73s
         3        3267.4562            7.01s
         4        3032.2269            6.95s
         5        2817.6489            7.12s
         6        2618.6400            6.73s
         7        2441.0653            6.44s
         8        2278.0638            6.25s
         9        2128.2158            6.20s
        10        1994.3030            6.18s
        20        1149.6738            5.07s
        30         806.5839            3.88s
        40         641.4368            2.89s
        50         544.0859            1.89s
        60         484.8555            0.94s
        70         428.3446            0.00s
      Iter       Train Loss   Remaining Time 
         1        3819.0255           16.24s
         2        3529.5422           13.21s
         3        3267.4562           12.85s
         4        3032.2269           13.45s
        

gradient boosting with gridsearch cv

In [ ]:
gss_grid_cv =  GroupShuffleSplit(n_splits = 10, train_size = 0.7, random_state = 42)

gb_grid_cv = GradientBoostingRegressor(random_state=42,verbose=1)
para={"learning_rate":[0.05,0.1,0.2],"n_estimators":[100,200,300,400],"max_depth":[5,6,7,8],"min_samples_split":[150,200],"criterion":["squared_error"],"min_samples_leaf":[50,80] }
train_index_grid_cv,val_index_grid_cv=next(gss_grid_cv.split(x_train, y_train, Engine))
# define search
search = GridSearchCV(gb_grid_cv, param_grid=para, scoring=['r2','neg_root_mean_squared_error'], cv=gss_grid_cv, refit='r2')
# execute search
Y_train=np.asarray(y_train).ravel()
result = search.fit(x_train[train_index_grid_cv,:], Y_train[train_index_grid_cv],groups=Engine[train_index_grid_cv])
# get the best performing model fit on the whole training set
best_model = result.best_estimator_
yhat = best_model.predict(x_train[val_index_grid_cv,:])
# evaluate the model
r2 = r2_score(Y_train[val_index_grid_cv], yhat)
rmse=root_mean_squared_error(Y_train[val_index_grid_cv],yhat)
print("r2: ",r2)
print("rmse: ",rmse)

In [ ]:
import pickle
pickle.dump(best_model, open('gradient_boosting_model.pkl','wb'))

gradient boosting with nested gridsearchcv

In [ ]:
gss_outer = GroupShuffleSplit(n_splits = 10, train_size = 0.7, random_state = 42) #n_splits refers to number of folds required

# enumerate splits
outer_results = {"r2":[],"rmse":[]}
for i,(train_ix, test_ix) in enumerate(gss_outer.split(x_train, y_train, engine)):
    # split data
    X_train, X_test = x_train.iloc[train_ix, :], x_train.iloc[test_ix, :]
    Y_train, Y_test = y_train.iloc[train_ix], y_train.iloc[test_ix]
    Engine=engine[train_ix]
    Y_train=np.asarray(Y_train).ravel()
    Engine=Engine.ravel()
    # configure the cross-validation procedure
    gss_inner =  GroupShuffleSplit(n_splits = 3, train_size = 0.7, random_state = 42)
    # define the model
    gb = GradientBoostingRegressor(random_state=42,verbose=1)
    para={"learning_rate":[0.05,0.1,0.2],"n_estimators":[70,100,200,300],"max_depth":[5,6,7,8],"min_samples_split":[150,200],"criterion":["squared_error"],"min_samples_leaf":[50,80] }

    # define search
    search = GridSearchCV(gb, param_grid=para, scoring=['r2','neg_root_mean_squared_error'], cv=gss_inner, refit='r2')
    # execute search
    result = search.fit(X_train, Y_train,groups=Engine)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    r2 = r2_score(Y_test, yhat)
    rmse=root_mean_squared_error(Y_test,yhat)
    # store the result
    outer_results["r2"].append(r2)
    outer_results["rmse"].append(rmse)
    # report progress
    print('>r2=%.3f, est=%.3f, cfg=%s' % (r2, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('R2: %.3f (%.3f)' % (np.mean(outer_results["r2"]), np.std(outer_results["r2"])))
print('RMSE: %.3f (%.3f)' % (np.mean(outer_results["rmse"]), np.std(outer_results["rmse"])))